In [84]:
import pandas as pd
import pandasql as ps

In [85]:
data_path = '../data/raw_data/atp_matches_68_24.csv'
data = pd.read_csv(data_path)

### Select unique winner and loser infos

In [86]:
df_winner = data[['winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age', 'tourney_date']]

# SQL query on the DataFrame
query = "SELECT winner_id, winner_name, winner_hand, winner_ht, winner_ioc, winner_age, max(tourney_date) " \
        "FROM df_winner " \
        "GROUP BY winner_id"
ps_winner = ps.sqldf(query, locals())

df_loser = data[['loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'tourney_date']]

# SQL query on the DataFrame
query = "SELECT loser_id, loser_name, loser_hand, loser_ht, loser_ioc, loser_age, max(tourney_date) " \
        "FROM df_loser " \
        "GROUP BY loser_id"
ps_loser = ps.sqldf(query, locals())
ps_loser

,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,max(tourney_date)
0,100001,Gardnar Mulloy,R,185.0,USA,63.2,19770207
1,100002,Pancho Segura,R,168.0,ECU,52.7,19740303
2,100003,Frank Sedgman,R,180.0,AUS,48.6,19760510
3,100004,Giuseppe Merlo,R,NaN,ITA,45.6,19730602
4,100005,Richard Gonzalez,R,188.0,USA,45.3,19730917
...,...,...,...,...,...,...,...
7408,212720,Juan Carlos Fuentes Vasquez,U,NaN,ESA,18.9,20240914
7409,212722,Kwok Shun Dasson Chan,U,NaN,HKG,18.5,20240203
7410,212733,Matthew Forbes,U,NaN,USA,18.3,20240826
7411,212866,Marcos Lee Chan Baratau,U,NaN,ECU,18.8,20240914


### Join tables

In [87]:
ps_winner = ps_winner.rename(columns={
    "winner_id": "id",
    "winner_name": "name",
    "winner_hand": "hand",
    "winner_ht": "height",
    "winner_ioc": "country",
    "winner_age": "age",
    "max(tourney_date)": "last_tourney_date"
})

ps_loser = ps_loser.rename(columns={
    "loser_id": "id",
    "loser_name": "name",
    "loser_hand": "hand",
    "loser_ht": "height",
    "loser_ioc": "country",
    "loser_age": "age",
    "max(tourney_date)": "last_tourney_date"
})

df_players = pd.concat([ps_winner, ps_loser], ignore_index=True)
df_players = df_players.sort_values(by='last_tourney_date', ascending=False)
df_unique_players = df_players.drop_duplicates(subset='id', keep='first')
players = df_unique_players.reset_index(drop=True)

### Save Player_DB

In [88]:
players.to_csv('../data/player/players_68_24.csv', index=False)